# In this notebook we are going to cover some of the most fundamentals concept of tensors using TensorFlow

More specifically, we're going to cover:

* introduction to tensors
* getting information from tensors
* manipulating tensors
* tensors and Numpy
* Using @tf.function (a way to speed up your regular Python Functions)
* Using GPUs with TensorFlow (or TPU's)
* Exercice for yourselves.

## Introduction to Tensors

### Creating tensors with `tf.constant()`

In [1]:
# import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.17.0


In [2]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# check the number of dimensions of a tensor (ndim for number of dimensions)
scalar.ndim

0

In [4]:
# create a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
# check the dimension of our vector
vector.ndim

1

In [6]:
# create a matrix (has more than one dimension)
matrix = tf.constant([[10, 8],[7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  8],
       [ 7, 10]], dtype=int32)>

In [7]:
matrix.ndim

2

In [8]:
# create another matrix
another_matrix = tf.constant([[10., 7.], [3., 2.],[8.,9.]], dtype=tf.float16) #specify the data type parameter
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [9]:
another_matrix.ndim

2

In [10]:
#let's create a tensor
tensor = tf.constant([[[1,2,4],[4,5,6]],[[7,8,9],[10,11,12]],[[13,14,15],[16,17,18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  4],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [11]:
tensor.ndim

3

What we've created so fat:
* Scalar: a single number
* vector: a number with direction (e.g., wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (when n can be any number)

### Creating tensor with `tf.Variable()`


In [12]:
# Create the same tensor with tf.Variable() as above -> variable tensor

In [13]:
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [14]:
# Let's try change one of the elements in our changeable tensor
changeable_tensor[0] = 7

TypeError: 'ResourceVariable' object does not support item assignment

In [ ]:
#How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

In [ ]:
# Let's try change on our unchangeable tensor
unchangeable_tensor[0].assign(7)
unchangeable_tensor

**Note**: Rarely in practice will you need to decide whether to use `tf.constant`or `tf.Variable`to create tensors aas TensorFlow does this for us. However in doubt, use `tf.constant`and change it later if needed

### Creating random tensors

In [15]:
# Lets create two random (but not the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproductibily
random_1 = random_1.normal(shape=(3,2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193765, -1.8107855 ]], dtype=float32)>

In [16]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))
random_2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193765, -1.8107855 ]], dtype=float32)>

**Note**: those tensors though seemingly random are not because of the seed(42)

### Shuffle a tensor

In [17]:
# Shuffling the order of elements in a tensor

In [18]:
not_shuffled = tf.constant([[10, 7], [3,4], [2,5]])
not_shuffled.ndim

2

In [19]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [20]:
# Shuffle our not shuffled tensor
tf.random.set_seed(42)
shuffled = tf.random.shuffle(not_shuffled, seed=42)
shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [21]:
# Creating tensors from Numpy arrays

### Turns Numpy arrays into tensors
The main difference between Numpy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing).

In [22]:
import numpy as np
numpy_A = np.arange(1,25, dtype=np.int32) # create a numpy array between 1 and 25

#X = tf.constant(some_matrix)
#y = tf.constant(vector)

A = tf.constant(numpy_A)

In [23]:
A

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [24]:
B = tf.constant(numpy_A, shape=(2,3,4))

In [25]:
B

<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]], dtype=int32)>

### Getting information from tensors
When dealing with tensors, we problably want to aware of:
*   Shape
*   Rank
*  Dimensions
* Size


In [26]:
# Shape
# create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2,3,4,5])
rank_4_tensor


<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [27]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [28]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)


(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [29]:
# Create various attributes of our tensor
print('Datatype of every elements:', rank_4_tensor.dtype)
print('Number of dimensions (rank):', rank_4_tensor.ndim)
print('Shape of tensor:', rank_4_tensor.shape)
print('Elements along the O axis:', rank_4_tensor.shape[0])
print('Elements along the last axis:', rank_4_tensor.shape[-1])
print('total number of elements in our tensor:', tf.size(rank_4_tensor).numpy())

Datatype of every elements: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the O axis: 2
Elements along the last axis: 5
total number of elements in our tensor: 120


### Indexing tensors

Tensors can be indexes just like Python lists

In [30]:
#get the firsts 2 elements of each dimensions
rank_4_tensor[:2,:2,:2,:2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [31]:
# get the first element from each dimension index except for the final one
rank_4_tensor[:1, :1,:1,:]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [32]:
# Create a rank-2 tensor
rank_2_tensor = tf.constant([[10, 4],[5, 10]])
rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  4],
       [ 5, 10]], dtype=int32)>

In [33]:
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [34]:
# Get the last item of each or our rank 2 tensor
rank_2_tensor[:,1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 4, 10], dtype=int32)>

In [35]:
# Add dimensions (useful when preparing data for NN when we need to mathc dimensions)
rank_3_tensor = rank_2_tensor[...,tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 4]],

       [[ 5],
        [10]]], dtype=int32)>

In [36]:
rank_2_tensor[tf.newaxis, ...].shape

TensorShape([1, 2, 2])

In [37]:
# Alternative to tf.newaxis
tf.expand_dimes(rank_2_tensor, axis=-1)# -1 means expands in the last axis

AttributeError: module 'tensorflow' has no attribute 'expand_dimes'

### Manipulation tensors (tensors operations)

***Basic operations***

* `+`, `-`, `*`, `/`

In [ ]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10,7],[3,4]])
tensor + 10

We can use the tensor flow built-in function too (like for Numpy)

In [38]:
# example
tf.multiply(tensor,10)

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 10,  20,  40],
        [ 40,  50,  60]],

       [[ 70,  80,  90],
        [100, 110, 120]],

       [[130, 140, 150],
        [160, 170, 180]]], dtype=int32)>

### Matrix mutliplication

In [39]:
# Matrix multiplication in tensorflow
tf.linalg.matmul(tensor, tensor)

InvalidArgumentError: {{function_node __wrapped__BatchMatMulV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [3,2,3], In[1]: [3,2,3] [Op:BatchMatMulV2] name: 

In [40]:
# exercice:
ten1 = tf.constant([[1,2,5],[7,2,1],[3,3,3]])
ten2 = tf.constant([[3,5],[6,7],[1,8]])

tf.linalg.matmul(ten1,ten2)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [41]:
# using Python matrix multiplicator:
ten1 @ ten2

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [42]:
# create a tensor
X = tf.constant(
    [[1,2],
 [3,4],
  [5,6]])

X2 = tf.constant(
    [[7,8],
 [9,10],
  [11,12]])
# we need to transpose in order to make the size match?
X2 = tf.transpose(X2)
X2

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  9, 11],
       [ 8, 10, 12]], dtype=int32)>

In [43]:
X @ X2

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

there are two rules to use matrix mutliplication:

1. the inner dimensions must match
2. the resulting matrix has the shape of the outer dimensions

### The dot product

Matrix mutliplocation is also referred to as the dot product.

You can perform matrix multiplication using:

* `tf.matmul()`
* `tf.tensordot()`


In [44]:
# Perform the dot product on X and X2 (with correct dimensions) using tf.tensordot()

In [45]:
tf.tensordot(X,X2, axes=0)

<tf.Tensor: shape=(3, 2, 2, 3), dtype=int32, numpy=
array([[[[ 7,  9, 11],
         [ 8, 10, 12]],

        [[14, 18, 22],
         [16, 20, 24]]],


       [[[21, 27, 33],
         [24, 30, 36]],

        [[28, 36, 44],
         [32, 40, 48]]],


       [[[35, 45, 55],
         [40, 50, 60]],

        [[42, 54, 66],
         [48, 60, 72]]]], dtype=int32)>

In [46]:
tf.tensordot(X,X2, axes=1)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

### Change the datatype of a tensor

In [47]:
# create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [48]:
C = tf.constant([7, 10])
C.dtype

tf.int32

In [49]:
# change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)

In [50]:
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [51]:
u = tf.constant([1,2,3])
v = tf.constant([4,5])

tf.tensordot(u,v, axes=0)


<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 4,  5],
       [ 8, 10],
       [12, 15]], dtype=int32)>

### Tensor aggregation (finding the min, max, mean & more)

Aggregating tensors = condesing them from multiple values down to a smaller amount of values.

In [53]:
# getting the absolute values:
D = tf.constant([-7, 10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([-7, 10], dtype=int32)>

In [54]:
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Let's go through the following form of aggregation:

* get the minimum
* get the maximum
* get the mean of a tensor
* get the sum of a tensor

In [57]:
# Creating a random tensor with values between 0 and 100 of size 50:

E = tf.constant(np.random.randint(0,100, size=50))
E.ndim, E.shape, tf.size(E)


(1, TensorShape([50]), <tf.Tensor: shape=(), dtype=int32, numpy=50>)

In [58]:
# find the minimum
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=1>

In [59]:
#find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=99>

In [60]:
# find the mean of a tensor
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=51>

In [61]:
#get the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2556>

In [67]:
# change datatype of the tensor
E = tf.cast(E, dtype=tf.float32)

In [70]:
#variance
tf.math.reduce_variance(E)

<tf.Tensor: shape=(), dtype=float32, numpy=875.7456>

In [69]:
# standard deviation
tf.math.reduce_std(E)

<tf.Tensor: shape=(), dtype=float32, numpy=29.592999>

**the position maximum and minimum of a tensor**

In [75]:
# Createt a new tensor for finding the positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50]
                      )
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [76]:
# find the positional maximum
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [77]:
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [78]:
# Find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [79]:
assert F[tf.argmax(F)] == tf.reduce_max(F)

In [80]:
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

### Squeezing the tensors (removing all single dimensions)

In [83]:
# create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1,1,1,1,50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [84]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [85]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>,
 TensorShape([50]))

### One hot encode tensors

One hot encoding means turning our data into tensors to be able to use them with our NN

In [1]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [2]:
#create a list of indices
some_list = [0, 1, 2, 3] #could be red, green, blue, purple

# one hot encore our list of indices:

tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [3]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value = 'yo I love deep learning', off_value = 'I also like to dance')

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'yo I love deep learning', b'I also like to dance',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'yo I love deep learning',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'yo I love deep learning', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'I also like to dance', b'yo I love deep learning']],
      dtype=object)>

### More tensor math operations

ok here use tensorflow and its built in math operations

* squaring
* log
* squre root


In [5]:
H = tf.range(1,10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [6]:
# square
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [8]:
#square root (require to cast into another type)
tf.math.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.2360678, 2.4494896,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [10]:
# log
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensors and Numpy

In [12]:
# create a tensor directly using Numpy
import numpy as np
J = tf.constant(np.array([3. , 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [14]:
# convert our tensor back to a NumPy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [15]:
#convert tensor J to a Numpy array as such:
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [17]:
# The default tupes of each array are slighlty different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.]
                       )
# check the data type of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Finding access to GPU

In [24]:
tf.config.list_physical_devices('GPU')
!nvidia-smi

Thu Aug 15 11:30:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0              29W /  70W |    105MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--